# Analysis of Blackjack Odds

Using Python to run simulations of blackjack with various card counting mechanisms to analyze odds

## Section 1: Infrastructure Code

This first section contains the implementations that we'll need to do our analysis. We'll build up classes for cards, decks, and different types of players that use various mechanisms for making decisions.

In [1]:
import collections
from typing import Dict, List

rank_to_value: Dict[str, int] = { str(rank): rank for rank in range(2, 11) }
# TODO: how to handle card value treating as Aces as worth 1 ?
rank_to_value.update({'J': 10, 'Q': 10, 'K': 10, 'A': 11})

CardBase = collections.namedtuple('Card', ['rank', 'suit'])

class Card(CardBase):

    def __str__(self):
        return f'{self.rank} of {self.suit}'

    @property
    def value(self) -> int:
        return rank_to_value[self.rank]

This FrenchDeck implementation comes from Luciano Ramalho's excellent book [Fluent Python](http://shop.oreilly.com/product/0636920032519.do)

In [2]:
from random import randrange

class FrenchDecks:
    """Represents one or more decks of cards

    >>> deck = FrenchDeck()
    >>> len(deck)
    52
    >>> five_decks = FrenchDeck(num_decks=5)
    >>> len(five_decks)
    260
    """

    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suit = 'spades diamonds clubs hearts'.split()

    def __init__(self, num_decks=1):
        self._cards = [Card(rank, suit)
                      for suit in self.suit
                      for rank in self.ranks
                      for _ in range(num_decks)]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, pos, card):
        self._cards[pos] = card
        
    def pop_random(self) -> Card:
        return self._cards.pop(randrange(len(self._cards)))

    def deal_one(self) -> Card:
        """Returns a single card and removes it from deck"""
        return self.pop_random()

    def deal_two(self) -> List[Card]:
        """Returns list of two cards from deck"""
        return [self.pop_random(), self.pop_random()]

In [3]:
class Hand:
    """Represents a Blackjack hand
    
    >>> hand = Hand([])
    >>> hand.get_possible_values()
    0
    >>> hand = Hand([Card('A', 'spades'), Card('A', 'hearts')])
    >>> hand.get_possible_values()
    [2, 12, 22]
    """

    def __init__(self, hand: List[Card]):
        self.hand = hand

    def __str__(self):
        return '\n'.join(str(c) for c in self.hand)

    def __getitem__(self, pos) -> Card:
        return self.hand[pos]
    
    def deal_card(self, card: Card):
        """Add card to hand"""
        self.hand.append(card)

    def deal_cards(self, cards: List[Card]):
        """Add multiple cards to hand"""
        self.hand.extend(cards)

    def get_possible_values(self) -> List[int]:
        """Gets the possible values of a hand, accounting for Aces as 1 or 11

        Returns list of length n+1 where n is number aces in hand
        """
        possible_values = [0]
        for card in self.hand:
            if card.rank == 'A':
                values_with_1 = [val +  1 for val in possible_values]
                values_with_11 = [val + 11 for val in possible_values]
                possible_values = list(set(values_with_1 + values_with_11))
            else:
                possible_values = [val + card.value for val in possible_values]

        return sorted(possible_values)

    @property
    def hands_less_than_equal_21(self) -> List[int]:
        """Return the possible values that hand could take but do not include busted hands"""
        return [val for val in self.get_possible_values() if val <= 21]

    @property
    def value(self) -> int:
        """Highest possible value in hand under 21

        Returns None if no values in hand under 21
        """
        return None if not self.hands_less_than_equal_21 else max(self.hands_less_than_equal_21)

    @property
    def soft(self) -> bool:
        """Returns whether this is a soft hand, i.e. more than one possible value under 21"""
        return len(self.hands_less_than_equal_21) >  1

    def busted(self) -> bool:
        """Returns whether hand has busted, all possible values over 21"""
        return len(self.hand) > 0 and len(self.hands_less_than_equal_21) == 0

In [4]:
import enum

class BlackjackMove(enum.Enum):
    HIT = 1
    STAY = 2
    DOUBLE_DOWN = 3
    SPLIT = 4

In [5]:
class Player:
    """Base class for representing a player two or more cards

    Implementation of get_move() should be left up to child classes
    """

    def __init__(self, hand: Hand):
        self.hand = hand

    def __repr__(self):
        return str(self.hand)

    def busted(self) -> bool:
        return self.hand.busted()

    def deal(self, card: Card):
        self.hand.deal_card(card)

    def get_move(self, dealer_card: Card) -> BlackjackMove:
        """TODO: best way to use this as base class and ensure derived classes implement?"""
        raise NotImplementedError

In [6]:
class SimplePlayer(Player):
    """Player who follows basic strategy"""

    def get_move(self, dealer_card: Card) -> BlackjackMove:
        """TODO: describe basic strategy

        TODO: implement splits and double downs
        """
        if self.hand.value < 12:
            return BlackjackMove.HIT
        elif self.hand.value == 12:
            if 4 <= dealer_card.value <= 6:
                return BlackjackMove.STAY
            else:
                return BlackjackMove.HIT
        elif 13 <= self.hand.value <= 16:
            if self.hand.soft:
                return BlackjackMove.HIT
            else:
                if dealer_card.value <= 6:
                    return BlackjackMove.STAY
                else:
                    return BlackjackMove.HIT
        else:
            return BlackjackMove.STAY

In [7]:
class Dealer(Player):

    def get_move(self) -> BlackjackMove:
        """Dealer hits on anything under 17 or on soft 17"""
        if self.hand.value < 17:
            return BlackjackMove.HIT
        elif self.hand.value == 17:
            if self.hand.soft:
                return BlackjackMove.HIT
            else:
                return BlackjackMove.STAY
        else:
            return BlackjackMove.STAY

In [8]:
from itertools import repeat

# Simple dealer vs player round
wins: Dict[str, int] = {'player': 0, 'dealer': 0}
for _ in repeat(None, 10**5):
    deck = FrenchDecks()
    dealer = Dealer(Hand(deck.deal_two()))
    player = SimplePlayer(Hand(deck.deal_two()))

    while True:
        player_move = player.get_move(dealer.hand[0])

        if player_move == BlackjackMove.STAY:
            break
        else:
            player.deal(deck.deal_one())
            if player.busted():
                break

    while True:

        dealer_move = dealer.get_move()

        if dealer_move == BlackjackMove.STAY:
            break
        else:
            dealer.deal(deck.deal_one())
            if dealer.busted():
                break

    if player.busted():
        wins['dealer'] += 1
    elif dealer.busted():
        wins['player'] += 1
    elif player.hand.value <= dealer.hand.value:
        wins['dealer'] += 1
    else:
        wins['player'] += 1

wins

{'player': 42745, 'dealer': 57255}